In [1]:
using LinearAlgebra, CSV, Tables

In [12]:
diagm(1:3)

3×3 Array{Int64,2}:
 1  0  0
 0  2  0
 0  0  3

In [19]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
# All close.
maximum(abs.(raw_data * raw_data .- data))

4.884981308350689e-15

In [4]:
selected_data = [3; 83; 85];
opt_inds = [79; 80; 81; 84];
k = 7

7

In [34]:
s = maximum(svd(raw_data[:, selected_data]).S)
u = svd(raw_data[:, selected_data]).U[:, 1];
v = svd(raw_data[:, selected_data]).V[:, 1];
D = Matrix(1.0I, 101, 101)[:, selected_data];
s

1.4499823628525759

In [25]:
svd(raw_data[:, selected_data]).S .^ 2

3-element Array{Float64,1}:
 2.102448852583539
 0.8291308235464361
 0.06842032387002067

In [26]:
maximum(eigvals(data))

25.582479205536032

In [28]:
maximum(eigvals(data - raw_data' * u * u' * raw_data))

19.502666782823834

In [38]:
eigvals(raw_data * (I - raw_data * D * v * v' * D' * raw_data' / s^2) * raw_data)

101-element Array{Float64,1}:
 -1.6703904757439067e-16
  0.0006295386766785218
  0.0008072406973032147
  0.0011094562996774356
  0.0013002649211967985
  0.0017708921303124247
  0.0020325222834940778
  0.0027270540894054657
  0.003795475645012561
  0.0041128626790023996
  ⋮
  1.8187821036058793
  2.7964453404400422
  2.9928365348369406
  3.3818398348729577
  4.416358704734597
  6.097704924714852
  7.936466991262928
  9.361963079175172
 19.5026667828238

In [ ]:
eigvals((I - raw_data * D * v * v' * D' * raw_data' / s^2) * data)

101-element Array{Float64,1}:
 -2.2162246791624813e-15
  0.0006295386766789598
  0.0008072406973028526
  0.0011094562996780684
  0.0013002649211963262
  0.0017708921303125737
  0.0020325222834935114
  0.002727054089405773
  0.003795475645012731
  0.004112862679002526
  ⋮
  1.8187821036058733
  2.7964453404400427
  2.992836534836933
  3.3818398348729537
  4.416358704734595
  6.097704924714907
  7.936466991262927
  9.361963079175153
 19.502666782823788